<a href="https://colab.research.google.com/github/Pavun-KumarCH/AI-Enhanced-RAG-System-for-Automated-University-Course-Content-Generation/blob/main/Multi_Search_Agentic_RAG_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Search Agentic-RAG System
### A system leveraging multiple knowledge retrieval tools for enhanced query processing.

### Framework: LangChain  
### LLM: OpenAI Models

---

## 1. Tools Used

### 1.1. Wiki
**Medium:** Wikipedia API or LangChain Wiki API integration  
Wiki is used to retrieve factual and broad knowledge across a variety of domains. It is especially effective for general knowledge queries or historical data, where a detailed explanation or background information is necessary.

### 1.2. arxiv
**Medium:** arxiv API for research paper retrieval  
arxiv provides access to scientific papers, useful for academic queries, technical topics, or research-related requests. This allows the system to respond with cutting-edge research or deep technical insights from recent publications.

### 1.3. Retriever
**Medium:** LangChain's Retriever Components  
The retriever serves as a tool for fetching relevant documents based on the embeddings of the query. It operates across multiple sources (Wiki, arxiv, etc.) and ensures that the LLM has the most relevant context for generating responses.

---

## 2. LLM and LangChain Integration

### 2.1. LLM: OpenAI Models
**Medium:** OpenAI GPT-4 and fine-tuned variations via LangChain  
The LLM generates responses based on the documents retrieved by the retriever. It can summarize, interpret, and produce detailed answers, factoring in the specific knowledge drawn from the sources (Wiki, arxiv) used by the retriever.

### 2.2. LangChain Framework
**Medium:** Chains and Agents for multi-modal data flow  
LangChain orchestrates the system’s flow, handling how user queries are split between different tools. It manages which tool (Wiki, arxiv, etc.) is used based on the nature of the question and ensures that the retrieved data is processed properly by the LLM.

- **Agentic Behavior:**  
  Dynamic agents enable the system to act autonomously, selecting the best retriever based on context and intelligently processing multi-tool requests.

---

## 3. Future Enhancements

### 3.1. Integration of Additional Data Sources
**Medium:** APIs or Web Scraping via LangChain  
Potential additions include sources like Google Scholar, GitHub, or other specialized databases that can enhance the system’s versatility across domains.

### 3.2. Fine-tuning for Specific Tasks
**Medium:** Custom fine-tuning of the LLM  
Fine-tuned models could be introduced to optimize the system for niche tasks, such as legal research or medical information retrieval.

# Requirements

In [52]:
!pip install --q arxiv wikipedia langchain_community langchain_openai faiss-cpu

In [45]:
#@title Load Dependencies
import os
from langchain import hub
from IPython.display import display, Markdown

from langchain_community.tools import WikipediaQueryRun, ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain.tools.retriever import create_retriever_tool

from langchain.agents import create_openai_tools_agent, AgentExecutor

# Load Environment Variable
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")

# Tools

In [39]:
#@title Wikipedia Tool
api_wrapper = WikipediaAPIWrapper(top_k_results = 1, doc_content_chars_max = 200)

wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

wiki_tool.name

'wikipedia'

In [21]:
#@title Arxiv Tool
# the website were all the research papers are being uploaded

arxiv_api_wrapper = ArxivAPIWrapper()
arxiv_tool = ArxivQueryRun(api_wrapper = arxiv_api_wrapper)

arxiv_tool.name

'arxiv'

In [17]:
# Retriever Vectorstore

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200).split_documents(docs)
vector_db = FAISS.from_documents(documents, OpenAIEmbeddings())

retriever = vector_db.as_retriever()

retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x78a690a45e10>, search_kwargs={})

In [19]:
#@title Retriever Tool
retrival_tool = create_retriever_tool(retriever, "langsmith_search", "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

retrival_tool.name

'langsmith_search'

In [40]:
tools =  [wiki_tool, arxiv_tool, retrival_tool]

# Agents

In [41]:
#@ Agent

# Initialize the LLM Model
llm = ChatOpenAI(
    model = "gpt-3.5-turbo-0125",
    temperature = 0.3,
    top_p = 0.8,
    max_tokens = 400,
)

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# Intializing Agent
Agent = create_openai_tools_agent(llm = llm,
                                  tools = tools,
                                  prompt = prompt)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


## Agent Executor

In [43]:
# Executor
agent_executor = AgentExecutor(agent = Agent, tools = tools, verbose = True)

# Response's

In [49]:
#@title Retriver Tool Invoke
question = {"input" : "Tell me About LangSmith and LangSmith Graph Agents"}
response = agent_executor.invoke(question)

# Render using Markdown
Markdown(response['output'])



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith'}`


Get started with LangSmith | ü¶úÔ∏èüõ†Ô∏è LangSmith

Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith‚ÄãPythonTypeScriptpip install -U langsmithyarn add langsmith2. Create an API key‚ÄãTo create an API key head to the Settings page. Then click Create API Key.3. Set up your environment‚ÄãShellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. L

LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows you to closely monitor and evaluate your application so you can ship quickly and with confidence. LangSmith works independently, and the use of LangChain is not necessary for its operation.

LangSmith provides tools for tracing and evaluating applications built on LangChain. It offers features like creating datasets, examples, and evaluators to test and monitor the performance of your applications. You can log traces to LangSmith using various methods and APIs provided by the platform.

If you have any specific questions or need more information about LangSmith or LangSmith Graph Agents, feel free to ask!

In [50]:
#@title Wikipedia Tool Invoke
question = {"input" : "Tell me About GAN"}
response = agent_executor.invoke(question)

# Render using Markdown
Markdown(response['output'])



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Generative Adversarial Network'}`


Page: Generative adversarial network
Summary: A generative adversarial network (GAN) is a class of machine learning frameworks and a prominent framework for approaching generative artificial intelligeA Generative Adversarial Network (GAN) is a class of machine learning frameworks and a prominent framework for approaching generative artificial intelligence. GANs consist of two neural networks, a generator, and a discriminator, which are trained simultaneously through adversarial training. The generator generates new data instances, while the discriminator evaluates the generated samples for authenticity. This process helps the generator improve its ability to create realistic data, leading to the generation of high-quality synthetic data.

> Finished chain.


A Generative Adversarial Network (GAN) is a class of machine learning frameworks and a prominent framework for approaching generative artificial intelligence. GANs consist of two neural networks, a generator, and a discriminator, which are trained simultaneously through adversarial training. The generator generates new data instances, while the discriminator evaluates the generated samples for authenticity. This process helps the generator improve its ability to create realistic data, leading to the generation of high-quality synthetic data.

In [51]:
#@title Arxiv Tool Invoke
question = {"input" : "What's the paper 1605.08386 about?"}
response = agent_executor.invoke(question)

# Render using Markdown
Markdown(response['output'])



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
allowed moves of arbitrary length. We show that the diameter of these graphs on
fibers of a fixed integer matrix can be bounded from above by a constant. We
then study the mixing behaviour of heat-bath random walks on these graphs. We
also state explicit conditions on the set of moves so that the heat-bath random
walk, a generalization of the Glauber dynamics, is an expander in fixed
dimension.The paper with identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch. The paper discusses the study of graphs on lattice points where edges are derived from a finite set of allowed moves of arbitrary length. The authors show that the diameter of these gra

The paper with identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch. The paper discusses the study of graphs on lattice points where edges are derived from a finite set of allowed moves of arbitrary length. The authors show that the diameter of these graphs on fibers of a fixed integer matrix can be bounded by a constant. The paper also explores the mixing behavior of heat-bath random walks on these graphs and provides conditions on the set of moves for the heat-bath random walk to be an expander in a fixed dimension.